In [ ]:
from funcs.WLANFuncs import *
from funcs.HelperFuncs import *

from multiprocessing import Pool
import os
import pandas as pd
from glob import glob

%matplotlib inline

pd.options.display.float_format = '{:6f}'.format

import warnings
warnings.filterwarnings("ignore")


## Setting configs


In [ ]:
# Directory storing the CSI and WLAN captures
resources_dir = "/Volumes/tim_details/tim_honours/CAPTURES"

# Directory to save plots to
plt_dir = "/Users/timothylee/Desktop/Uni/Yr5/Honours/honours_thesis/figures/plt_figs/"

# Supress pd scientific notation
pd.set_option('display.float_format', '{:.6f}'.format)

# Resolution of plots
plt.rcParams["figure.dpi"] = 100 # 300

# Backend to generate plots
# mpl.use("agg")
# %matplotlib ipympl
%matplotlib inline

# plt figure style
fig_style = "seaborn-v0_8-whitegrid"

# colormaps
cmap_qual = plt.get_cmap("Pastel1")
cmap_seq = plt.get_cmap("viridis")
cmap_cycl = plt.get_cmap("twilight")


## Convert JSON to H5
good for ML, similar to csv, and efficiently stored with correct dtypes

### Also adding derivate features
* Is upstream/downstream

In [ ]:
procs = 4

ssid = "67Marlie"
pwd = "sayplease"

dirs = glob("/Volumes/tim_details/tim_honours/CAPTURES/*/*")

for dir in dirs:
    # Setting up dirs
    clean_dir_junk(os.path.join(dir, "wlan"))
    make_dir(os.path.join(dir, "wlan_json"))
    make_dir(os.path.join(dir, "wlan_h5"))
    # Running multiprocessing: wlan pcap to h5
    starargs = [(dir, get_name(i)) for i in os.listdir(os.path.join(dir, "wlan"))]
    with Pool(procs, initializer=wlan_to_df_init_mp, initargs=(ssid, pwd)) as pool:
        pool.starmap(wlan_to_df_mp, starargs)
    # Cleaning dirs (removing json dir)
    remove_dir(os.path.join(dir, "wlan_json"))

In [ ]:
fp = "/Volumes/tim_details/tim_honours/CAPTURES/client_pc_50/v=A3gUpodXMv0/wlan_h5/cap_15.h5"
display(pd.read_hdf(fp, key=H5_WLAN_KEY, mode="r"))

## Investigating streams

## Visualising cumulative bytes by time (binned in time intervals)

In [ ]:
# dirs = glob("/Users/timothylee/Desktop/Uni/Yr5/Honours/honours_dev/2_analyse/wlan_captures")
dirs = glob("/Volumes/tim_details/tim_honours/CAPTURES/*/*")

for dir in dirs:
    # Getting directory and file paths
    h5_dir = os.path.join(dir, "wlan_h5")
    plot_dir = os.path.join(dir, "wlan_plots")
    plot_fp = os.path.join(plot_dir, "all_cumsum.png")
    # Preparing directories
    make_dir(plot_dir)

    # If the plot already exists, then skip
    if os.path.isfile(plot_fp):
        continue

    # Making plots
    fig, ax = WLANFuncs.wlan_cumsum_plot(h5_dir)
    fig.savefig(plot_fp)
    fig.clf();
plt.close();